In [1]:
%pip install azure-storage-blob
from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import pandas as pd
import re
%pip install scipy
%pip install scikit-learn
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
%pip install opencv-python
import cv2


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need

In [2]:
#set up storage for reading meta data
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "publicdata"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [3]:
# Loop through all blobs and process JSON files
json_data = {}

blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".json"):
        # Get the blob content
        blob_client = container_client.get_blob_client(blob.name)
        blob_content = blob_client.download_blob().content_as_text()

        # Convert to JSON and add to the dictionary using the blob's name as the key
        json_data[blob.name] = json.loads(blob_content)  # Now storing JSON content as a dictionary

# #Assigning specific JSON data to variables
# category = json_data.get("v1.0-mini/category.json", [])
# sensor = json_data.get("v1.0-mini/sensor.json", {})
# surface_ann = json_data.get("v1.0-mini/surface_ann.json", {})
# attribute = json_data.get("v1.0-mini/attribute.json", {})
# log = json_data.get("v1.0-mini/log.json", {})
# calibrated_sensor = json_data.get("v1.0-mini/calibrated_sensor.json", {})
# sample_data = json_data.get("v1.0-mini/sample_data.json", {})
# sample = json_data.get("v1.0-mini/sample.json", {})
# ego_pose = json_data.get("v1.0-mini/ego_pose.json", {})
# object_ann = json_data.get("v1.0-mini/object_ann.json", {})

default_path_train = "all-metadata/v1.0-train/"
default_path_test = "all-metadata/v1.0-test/"
default_path_val = "all-metadata/v1.0-val/"


def load_jsons(default_path):
    object_ann = json_data.get(default_path + "object_ann.json", {})
    sample_data = json_data.get(default_path + "sample_data.json", {})
    category = json_data.get(default_path + "category.json", [])

    return object_ann, sample_data, category

object_ann_train, sample_data_train, category_train = load_jsons(default_path_train)
#object_ann_test, sample_data_test, category_test = load_jsons(default_path_test)
#object_ann_val, sample_data_val, category_val = load_jsons(default_path_val)
#object_ann_test, sample_data_test, category_test

# Category Clustering and Label Encodeing

### Convert to dataframe for better processing

In [4]:
def json_to_df(object_ann, sample_data, category):
    #turn into proper json
    object_ann_json = json.dumps(object_ann, indent=4)
    sample_data_json = json.dumps(sample_data, indent=4)
    category_json = json.dumps(category, indent=4)
    #turn into dataframe
    object_ann_df = pd.read_json(object_ann_json)
    category_df = pd.read_json(category_json)
    sample_data_df = pd.read_json(sample_data_json)
    return object_ann_df, sample_data_df, category_df

In [5]:
object_ann_df_train, sample_data_df_train, category_df_train = json_to_df(object_ann_train, sample_data_train, category_train)

In [6]:
object_ann_df_train, sample_data_df_train, category_df_train = json_to_df(object_ann_train, sample_data_train, category_train)
#object_ann_df_test, sample_data_df_test, category_df_test = json_to_df(object_ann_test, sample_data_test, category_test)
#object_ann_df_val, sample_data_df_val, category_df_val = json_to_df(object_ann_val, sample_data_val, category_val)
#object_ann_df_test.shape, sample_data_df_test.shape, category_df_test.shape

### Cluster chosen Categories into right "Parent" - category

In [7]:
#clustering of categories
def transform_category(category_str):
    if re.match(r'^human\.', category_str):
        return 'Human'
    if re.match(r'^movable_object\.barrier', category_str):
        return 'Barrier'
    if re.match(r'^movable_object\.cone', category_str):
        return 'Cone'
    if re.match(r'^vehicle\.bicycle', category_str):
        return 'Bike'
    if re.match(r'^vehicle\.motorcycle', category_str):
        return 'Motorcycle'
    if re.match(r'^vehicle\.truck', category_str):
        return 'Truck'
    if re.match(r'^vehicle\.car', category_str):
        return 'Car'
    if re.match(r'^movable_object\.trafficcone', category_str):
        return 'Trafficcone'
    return None

def category_proc(category_df):
    #apply function to the category column -> clustering of chosen categories
    category_df['name'] = category_df["name"].apply(transform_category)

    #remove description
    category_df = category_df.drop(columns="description")

    #drop rows that are not included in the chosen categories
    category_df = category_df.dropna(subset=['name'])
    return category_df

In [8]:
category_df_train_new = category_proc(category_df_train)
#category_df_test = category_proc(category_df_test)
#category_df_val = category_proc(category_df_val)
category_df_train

,token,name,description
0,63a94dfa99bb47529567cd90d3b58384,None,"All animals, e.g. cats, rats, dogs, deer, birds."
1,a86329ee68a0411fb426dcad3b21452f,None,Surfaces should be regarded with no concern of...
2,1fa93b757fc74fb197cdd60001ad8abf,Human,Adult subcategory.
3,b1c6de4c57f14a5383d9f963fbdcb5cb,Human,Child subcategory.
4,909f1237d34a49d6bdd27c2fe4581d79,Human,Construction worker
5,403fede16c88426885dd73366f16c34a,Human,"A small electric or self-propelled vehicle, e...."
6,e3c7da112cd9475a9a10d45015424815,Human,Police officer.
7,6a5888777ca14867a8aee3fe539b56c4,Human,"Strollers. If a person is in the stroller, inc..."
8,b2d7c6c701254928a9e4d6aac9446d79,Human,"Wheelchairs. If a person is in the wheelchair,..."
9,653f7efbb9514ce7b81d44070d6208c1,Barrier,Temporary road barrier placed in the scene in ...


### Merging Dataframes based on Data model and FK-PK dependencies & proper encoding

In [13]:

def df_merging_encoding(object_ann_df, sample_data_df, category_df):

    label_data_v2 = None

    #merge dataframes based on foreign keys to connect labeling with image data
    obj_cat = pd.merge(object_ann_df, category_df, left_on='category_token', right_on='token', how='inner')
    obj_cat = obj_cat.dropna(subset=['name'])

    #remove annotations, where bboxes are too small -> after resizing not properly visible anymore
    

    #remove list of columns from dataframe
    columns_to_remove = ["token_x", "token_y", "category_token", "bbox", "mask", "attribute_tokens", ]
    obj_cat = obj_cat.drop(columns=columns_to_remove)

    #filter so that only key frames are included
    sample_data_df = sample_data_df[sample_data_df["is_key_frame"] == True]

    #merge with sample data
    label_data_v1 = pd.merge(sample_data_df, obj_cat, left_on='token', right_on='sample_data_token', how='left')
    label_data_v1['name'] = label_data_v1['name'].fillna('empty')
    #group to see all labels for each image
    label_data_v1 = label_data_v1.groupby("filename")["name"].value_counts().rename("count").reset_index()
    
    
    label_data_v1.to_excel("labeled.xlsx")

    #getting files with more than 5 annotations
    
    # files_to_drop = label_data_v1.groupby("filename")["count"].sum().reset_index()
    # files_to_drop = files_to_drop[files_to_drop["count"] > 5]

    #turn labels from names to label_data_v1 variables
    dummies = pd.get_dummies(label_data_v1['name'])



    # Join the dummy variables to the main dataframe
    labeled = pd.concat([label_data_v1, dummies], axis=1)
    labeled[dummies.columns] = labeled[dummies.columns].astype(int)

    # Drop the original 'name' column
    labeled = labeled.drop(['name'], axis=1)
    labeled = labeled.drop(['empty'], axis=1)
    labeled = labeled.drop(['count'], axis=1)

    # Group by 'filename' and aggregate the data
    labeled = labeled.groupby('filename').agg({
        'Human': 'max',
        'Barrier': 'max',
        'Bike': 'max',
        'Motorcycle': 'max',
        'Truck': 'max',
        'Car': 'max',
        'Trafficcone': 'max'
    }).reset_index()

    #removing the images with more than 5 annotations
    
    # labeled = labeled[~labeled['filename'].isin(files_to_drop['filename'])]
    # print("Shape: ", labeled.shape)
    
    #labeled = labeled.drop(columns="Total")
    labeled.to_excel("labeled.xlsx")

    labeled['Total'] = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone"]].sum(axis=1)
    #labeled['NoDetec'] = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone"]].apply(lambda row: 1 if (row == 0).all() else 0, axis=1)
    return labeled

In [14]:
labeled_train = df_merging_encoding(object_ann_df_train, sample_data_df_train, category_df_train)
labeled_train
#print("XXX", object_ann_df_test)
#labeled_test = df_merging_encoding(object_ann_df_test, sample_data_df_test, category_df_test)
#labeled_val = df_merging_encoding(object_ann_df_val, sample_data_df_val, category_df_val)

,filename,Human,Barrier,Bike,Motorcycle,Truck,Car,Trafficcone,Total
0,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,1,4
1,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,1,1,0,4
2,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,0,1,1,0,3
3,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,0,3
4,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,1,1,0,1,1,5
...,...,...,...,...,...,...,...,...,...
67274,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,1,0,0,1,1,0,4
67275,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,0,0,1,0,2
67276,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,1,0,1,0,3
67277,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,0,0,0,0,0,0,1,1


### Deleting images that have low quality

In [15]:
#use nicks code


df = labeled_train
print("Shape pre: ", df.shape)

# Function to calculate average brightness
def calculate_brightness(image):
    grayscale_image = image.convert("L")
    pixel_values = np.array(grayscale_image)
    return np.mean(pixel_values)

# Function to calculate contrast
def calculate_contrast(image):
    grayscale_image = image.convert("L")
    stat = ImageStat.Stat(grayscale_image)
    return stat.stddev[0]

# Create a list to store filenames that need to be removed
files_to_remove = []

# Process each blob
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".jpg") and blob.name.startswith("samples/"):
        #print(f"Processing {blob.name}")
        blob_client = container_client.get_blob_client(blob.name)
        blob_data = blob_client.download_blob().readall()
        image_data = io.BytesIO(blob_data)
        image = Image.open(image_data)

        # Calculate brightness and contrast
        brightness = calculate_brightness(image)
        contrast = calculate_contrast(image)

        #print(f"Brightness: {brightness}, Contrast: {contrast}")

        # Check if the image meets the removal criteria
        if brightness < 60 or contrast < 30:
            files_to_remove.append(blob.name)

# Remove rows from the DataFrame
filtered_df = df[~df['filename'].isin(files_to_remove)]
print("Shape post: ", filtered_df.shape)


# Save the filtered DataFrame back to the CSV file
filtered_df.to_csv("cleaned_preprocessed.csv", index=False)

Shape pre:  (67279, 9)
Shape post:  (67277, 9)


## Addressing imbalance

#### Status quo of imbalance currently:

In [ ]:
lab_no_barrier = labeled_train[[]]

# Calculating label counts
label_counts = labeled_train.iloc[:, 1:7].sum()
max_count = label_counts.max()

# Calculating IRLbl
ir_lbl = max_count / label_counts

# Calculating MeanIR
mean_ir = ir_lbl.mean()

# Calculating MaxIR
max_ir = ir_lbl.max()

# Calculating CVIR
cv_ir = ir_lbl.std() / mean_ir

# Displaying the results
ir_lbl, mean_ir, max_ir, cv_ir

(Human          1.515131
 Barrier       12.322205
 Bike           3.686643
 Motorcycle     4.715092
 Truck          3.552944
 Car            1.000000
 dtype: float64,
 4.465335929888663,
 12.3222049689441,
 0.9176040175824948)

#### Coping with imbalance:

In [ ]:
# print(labeled_train.shape)

# # Normalize the IRLbl to get sampling probabilities inversely proportional to their frequency
# sampling_weights = 1 / ir_lbl
# sampling_weights /= sampling_weights.sum()

# # Calculate how many samples you might want to add to each label
# additional_samples_needed = (sampling_weights * 1000000).astype(int)  # scale factor of 1000 for example

# # Function to oversample data
# def oversample_data(df, label, n_samples):
#     # Filter the dataframe for samples containing the specific label
#     df_label = df[df[label] == 1]
#     # Resample the dataframe
#     return df_label.sample(n=n_samples, replace=True)

# # Create an empty DataFrame to hold oversampled data
# oversampled_data = pd.DataFrame(columns=labeled_train.columns)

# # Oversample for each label based on additional samples needed
# for label, n_samples in additional_samples_needed.items():
#     if n_samples > 0:  # Check if oversampling is needed
#         oversampled_data = pd.concat([oversampled_data, oversample_data(labeled_train, label, n_samples)])

# # Combine the original DataFrame with the oversampled DataFrame
# df_oversampled = pd.concat([labeled_train, oversampled_data]).reset_index(drop=True)

# df_oversampled

In [ ]:
# # Calculating label counts
# label_counts = df_oversampled.iloc[:, 1:7].sum()
# max_count = label_counts.max()

# # Calculating IRLbl
# ir_lbl = max_count / label_counts

# # Calculating MeanIR
# mean_ir = ir_lbl.mean()

# # Calculating MaxIR
# max_ir = ir_lbl.max()

# # Calculating CVIR
# cv_ir = ir_lbl.std() / mean_ir

# # Displaying the results
# ir_lbl, mean_ir, max_ir, cv_ir

In [ ]:
#relative_occurrence = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone", "Total"]].mean()

#relative_occurrence


## Check for potential bias

In [ ]:
# df = labeled
# columns = ['Human', 'Barrier', 'Bike', 'Motorcycle', 'Truck', 'Car', 'Trafficcone']
# results = []

# # Nested loops to compute the Chi-squared test for each pair of variables
# for col1 in columns:
#     for col2 in columns:
#         if col1 != col2:
#             # Create a contingency table
#             contingency_table = pd.crosstab(df[col1], df[col2])
#             # Perform the chi-squared test
#             chi2, p_value, _, _ = chi2_contingency(contingency_table)
#             # Store results
#             results.append({'Variable 1': col1, 'Variable 2': col2, 'Chi-squared': chi2, 'p-value': p_value})

# # Convert results to DataFrame
# result_df = pd.DataFrame(results)
# result_df

A p-value above 0.05, suggests no significant associations and thus no apparent bias or dependency among the variables tested in the dataset.
Therefore, only the combination of Car & Truck (and vice versa) shows a dependency which might result in a bias in the models that will be trained based on this data.

This fact should be kept in mind when proceeding with the evaluation of the models.


# Preprocess data: Load, Normalized, Resize, ...

In [ ]:
def prepare_proc(labeled):

    shape1 = labeled.shape[0]
    print(shape1)
    #remove rows with more than 5 labels: too many labels
    labeled = labeled[labeled['Total'] < 5]

    shape2 = labeled.shape[0]
    print(shape2)

    removed_rows = shape1 - shape2
    print("Count of removed rows: ", removed_rows)

    labeled = labeled.drop('Total', axis=1)


    order_of_labels = ["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone"]

    #put lables into list
    labeled['labels'] = labeled[order_of_labels].values.tolist()

    #drop unnecessary columns - stored in array
    labeled = labeled.drop(columns=order_of_labels)
    return labeled

In [ ]:
labeled_train = prepare_proc(labeled_train)
labeled_train.to_csv("labeled_train.csv", index=False)
#labeled_test = prepare_proc(labeled_test)
#labeled_val = prepare_proc(labeled_val)

33182


KeyError: 'Total'

### Definition of preprocessing function

In [ ]:
def load_image(path):
    blob_client = container_client.get_blob_client(path)
    blob_data = blob_client.download_blob().readall()  # Directly read all bytes
    image = io.BytesIO(blob_data)
    return image  #returning the PIL Image object

def preprocess(image):
    image = Image.open(image)

    #convert the PIL image to a NumPy array
    image_array = np.array(image)

    grayscale_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)

    #resize the image
    resized_image = cv2.resize(grayscale_image, (256, 256))

    #apply histogram equalization to improve contrast
    equalized_image_8bit = cv2.equalizeHist(resized_image)

    # Re-normalize to [0, 1] range
    equalized_normalized_image = equalized_image_8bit / 255.0

    #apply histogram equalization
    #equalized_image = cv2.equalizeHist(np.uint8(normalized_image * 255))

    #apply edge detection using Canny
    #edges = cv2.Canny(np.uint8(normalized_image * 255), canny_threshold1, canny_threshold2)

    return equalized_normalized_image


def preprocessing(path):
    image = load_image(path)
    preprocessed = preprocess(image)
    return preprocessed


In [ ]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "datacomplete"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
def apply_preprocessing(labeled, purpose):

    labeled["image"] = labeled["filename"].apply(preprocessing)
    df.drop("filename", axis=1, inplace=True)
    json_data = labeled.to_json()

    path = 'ALL_'+ purpose + '_labeled_images.json'

    # Save JSON data to a file
    with open(path, 'w') as file:
        file.write(json_data)   


In [ ]:
apply_preprocessing(labeled_train, "train")
#apply_preprocessing(labeled_test, "test")
#apply_preprocessing(labeled_val, "val")